# Load the data

In [2]:
%load_ext autotime

import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from heapq import nlargest
import warnings
warnings.filterwarnings('ignore')

pd.set_option("display.max_colwidth",200)
pd.set_option("display.max_columns",20)
pd.set_option('float_format', '{:.3f}'.format)

In [1]:
# # load raw
# with open('App_usage_trace.txt') as f:
#     lines = f.readlines()#[:100000]
#     df = []
#     for line in lines:
#         data = line.split()
#         df.append(data)
# usage = pd.DataFrame(df, columns=['uid','timestamp', 'loc', 'app_id','traffic'])
# # output_users(usage)
# usage['traffic'] = usage['traffic'].astype('float64') / 1e6 # Convert traffic to MB
# usage['timestamp'] = usage['timestamp'].apply(lambda x: datetime.strptime(x, "%Y%m%d%H%M%S"))
# # Convert to datetime object
# usage['loc'] = usage['loc'].astype('int64')

In [3]:
# load cleaned
# with open('Cleaned_usage.txt') as f:
#     lines = f.readlines()#[:100000]
#     df = []
#     for line in lines:
#         line = line.replace('\n', '')
#         data = line.split(',')
#         df.append(data)
# usage = pd.DataFrame(df, columns=['uid','timestamp', 'loc', 'app_id','traffic'])
# # output_users(usage)
# # usage['traffic'] = usage['traffic'].astype('float64') / 1e6 # Convert traffic to MB
# usage['timestamp'] = usage['timestamp'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S')) 
# # usage['timestamp'] = usage['timestamp'].apply(lambda x: datetime.strptime(x, "%Y%m%d%H%M%S")) # Convert to datetime object
# # usage['timestamp'] = usage['timestamp'].datetime
# usage['loc'] = usage['loc'].astype('int64')
with open('usaged_data_cleaned.txt') as f:
    lines = f.readlines()#[:100000]
    df = []
    for line in lines:
        data = line.split(",")
        df.append(data)
usage = pd.DataFrame(df, columns=['index','uid','timestamp', 'loc', 'app_id','traffic'])
usage.drop(columns=['index'])
usage['timestamp'] = usage['timestamp'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
usage["traffic"] = usage['traffic'].astype('float64')


time: 3min 2s


In [8]:
# def output_users(data):
#     # group data into users
#     grouped_users = data.groupby('uid')
#     for user, group in grouped_users:
#         # iterare each user and output to seperate file
#         group.to_csv('user_{}.txt'.format(user), header=False, index=False, sep=' ')

time: 41.5 ms


In [4]:
# show the period where the oscillation happens
def present_oscillation(uid=888, threshold=5, jump_occurence=10):
    # Nathan: replaced usage with usage_cleaned
    user_uid = usage[usage['uid'] == str(uid)]

    # Set the thresold of time gap allowed between movement
    delta_threshold = timedelta(seconds=threshold)

    # Create the columns of the next time and lcoation
    user_uid['next_loc'] = user_uid['loc'].shift(-1).fillna(0).astype('int')
    user_uid['next_timestamp'] = user_uid['timestamp'].shift(-1)

    # Keep only movements
    user_uid = user_uid[user_uid['loc'] != user_uid['next_loc']]

    # Get the time gaps between movements
    user_uid['time_delta'] = user_uid['next_timestamp'] - user_uid['timestamp']

    # Get teleports and count the teleports
    # Should be able to identify the return trips
    teleports = user_uid[user_uid['time_delta'] < delta_threshold]
    teleports['tele'] = list(zip(teleports['loc'], teleports['next_loc']))

    # # Group and count the same jump
    tele_counts = teleports['tele'].value_counts().reset_index()
    tele_counts.columns = ['jump', 'count']
    tele_counts = tele_counts[tele_counts['count'] > jump_occurence]

    # Get the jumps that has a reasonable number of occurence
    tele_counts = tele_counts[tele_counts['count'] > jump_occurence]
    # tele_counts['jump'] = tuple(tele_counts['jump'])

    #return the rows that has tele_count in the "tele" column, so it returns the rows where the oscillation happens
    return teleports[teleports['tele'].isin(tele_counts["jump"])]


time: 11.2 ms


In [21]:
def replace_bases(data, loc_list):
    print_count = 0
    for oscillation_period in loc_list:
        # get starting index
        start = oscillation_period[0]
        # get ending index of oscillation period
        end = oscillation_period[1]
        # slice dataset to get requests within oscilation period
        initial_osc = data[start:end+1]
        oscillation_slice = data.loc[start:end+1,'loc']
        # get the count of each base station. 
        #   rename axis and reset index to get row index as column
        base_counts = oscillation_slice.value_counts().rename_axis('loc').reset_index(name='counts')
        # get list of unique base stations in oscilation period
        base_stations = base_counts['loc'].tolist()
        # get inital count of each base station within the oscillation period
        base_station_counts = base_counts['counts'].tolist()
        # get the top 2 counts of base stations
        # top_2 = nlargest(2, base_station_counts)
        # get difference between the top 2 base stations
        cur_diff = 0
        # we only declare one base station as majority if it has a count greater than 5 to the next greatest
        i = 1
        while cur_diff < 5:
            # we take into account of the next surrounding requests outside of the oscillation period
            # this will continue to increase by 1 request at each side
            temp_oscillation_slice = data.loc[start - i :end + i,'loc']
            # get new count of base stations in oscillation period
            base_counts = temp_oscillation_slice.value_counts().rename_axis('loc').reset_index(name='counts')
            # output count as list
            base_station_counts = base_counts['counts'].tolist()
            # get the top 2 counts of base stations
            top_2 = nlargest(2, base_station_counts)
            if len(top_2) < 2:
                i += 1
                continue
            # get the difference
            cur_diff = top_2[0] - top_2[1]
            # icnrement search range
            i += 1
        
        # assign variable with the majority base station in oscillation period
        largest_station = base_stations[base_station_counts.index(max(base_station_counts))]

        # manipulate original dataframe to make changes perm 
        data.loc[start:end+1,'loc'] = largest_station
   
        if print_count % 500 == 0:
            print(oscillation_slice.value_counts())
            print()
            print(initial_osc)
            print()
            print(data[start:end+1])
        print_count += 1


time: 3.44 ms


In [6]:
def remove_oscillation(user_list, jump_occurances, request_threshold):
    print("Getting each users oscillation requests")
    oscillation_list = []
    for uid in user_list: 
        # get all oscillations from current user
        # set threshold as 10 seconds and atleast 5 oscillation pairs
        user_oscillations = present_oscillation(uid,jump_occurances,request_threshold)

        # Create the columns of the next time and lcoation between frequent pairs
        user_oscillations['next_timestamp'] = user_oscillations['timestamp'].shift(-1)
        user_oscillations['time_delta'] = user_oscillations['next_timestamp'] - user_oscillations['timestamp']

        # set delta threshold between oscillation periods
        delta_threshold = timedelta(seconds=300)
        # declare list to hold indicies of oscillation periods
        start = True
        for index, request in user_oscillations.iterrows():
            # set the first index
            if start:
                start_index = index
                start = False

            # if delta to next oscillation request is greater than threshold then indicate this is the last index of current period
            if request['time_delta'] > delta_threshold:
                # set end index for current period
                end_index = index
                # append tuple (starting index, ending index) of the oscillation period
                oscillation_list.append((start_index, end_index))
                # declare the next request will be the start of the next oscillation period
                start = True     
    # replace all osillations for current user to dataframe
    oscillation_list.sort(key=lambda x: x[0])
    return oscillation_list
    # print(oscillation_list[100:105])
    # print("Replacing base stations in oscillation")        
    # replace_bases(working_usage, oscillation_list)

time: 3.74 ms


In [19]:
user_list = usage['uid'].unique()

# copy orignal dataset
# this is for comparison purposes
jump_occurances = 5
request_threshold = 5
working_usage = usage.copy()
osc_list = remove_oscillation(user_list, jump_occurances, request_threshold)


KeyboardInterrupt: 

In [22]:
replace_bases(working_usage,osc_list)

8194    16
Name: loc, dtype: int64

   index uid           timestamp   loc app_id  traffic
0      0   0 2016-04-20 08:13:19  8194    361    0.002
1      1   0 2016-04-20 08:13:20  8194    361    0.002
2      2   0 2016-04-20 08:13:22  8194    361    0.004
3      3   0 2016-04-20 08:13:30  8194    361    0.001
4      4   0 2016-04-20 08:13:31  8194    361    0.002
5      5   0 2016-04-20 08:16:44  8194    361    0.002
6      6   0 2016-04-20 08:16:49  8194     31    0.002
7      7   0 2016-04-20 08:16:49  8194    360    0.002
8      8   0 2016-04-20 08:16:49  8194    612    0.002
9      9   0 2016-04-20 08:16:50  8194     31    0.003
10    10   0 2016-04-20 08:17:25  8194     31    0.005
11    11   0 2016-04-20 08:17:26  8194     31    0.003
12    12   0 2016-04-20 08:17:27  8194     31    0.003
13    13   0 2016-04-20 08:17:28  8194    360    0.002
14    14   0 2016-04-20 08:17:33  8194    361    0.001

   index uid           timestamp   loc app_id  traffic
0      0   0 2016-04-20 08:1

KeyboardInterrupt: 

In [8]:
# print_count = 0
# for start, end in osc_list:
#     if print_count % 5000 == 0:
#         print(working_usage[start:end+1])
#     print_count += 1

   index uid           timestamp   loc app_id  traffic
0      0   0 2016-04-20 08:13:19  8194    361    0.002
1      1   0 2016-04-20 08:13:20  8194    361    0.002
2      2   0 2016-04-20 08:13:22  8194    361    0.004
3      3   0 2016-04-20 08:13:30  8194    361    0.001
4      4   0 2016-04-20 08:13:31  8194    361    0.002
5      5   0 2016-04-20 08:16:44  8194    361    0.002
6      6   0 2016-04-20 08:16:49  8194     31    0.002
7      7   0 2016-04-20 08:16:49  8194    360    0.002
8      8   0 2016-04-20 08:16:49  8194    612    0.002
9      9   0 2016-04-20 08:16:50  8194     31    0.003
10    10   0 2016-04-20 08:17:25  8194     31    0.005
11    11   0 2016-04-20 08:17:26  8194     31    0.003
12    12   0 2016-04-20 08:17:27  8194     31    0.003
13    13   0 2016-04-20 08:17:28  8194    360    0.002
14    14   0 2016-04-20 08:17:33  8194    361    0.001
         index  uid           timestamp   loc app_id  traffic
174239  174239  202 2016-04-24 16:31:46  4849    349    0.

In [14]:
#  df_osc_list = pd.DataFrame(osc_list, columns=['start', 'end'])

time: 91.4 ms


In [15]:
# df_osc_list

,start,end
0,0,14
1,28,30
2,34,34
3,38,51
4,66,79
...,...,...
68660,4171693,4171697
68661,4171704,4171829
68662,4171838,4171839
68663,4171844,4171844


time: 44.5 ms


In [15]:
# # get list of all unqiue users in dataset
# user_list = usage['uid'].unique()

# # copy orignal dataset
# # this is for comparison purposes
# jump_occurances = 2
# request_threshold = 5
# working_usage = usage.copy()
# remove_oscillation(user_list, jump_occurances, request_threshold)

In [9]:
usage.iloc[3852720:3853143]

,uid,timestamp,loc,app_id,traffic
3852720,979,2016-04-21 15:03:34,8564,1606,0.0013
3852721,979,2016-04-21 15:03:36,2161,114,0.004419
3852722,979,2016-04-21 15:04:18,2161,114,0.001293
3852723,979,2016-04-21 15:04:21,8564,5,0.001729
3852724,979,2016-04-21 15:04:22,8564,5,0.05744
...,...,...,...,...,...
3853138,979,2016-04-21 15:28:07,2161,2,0.001602
3853139,979,2016-04-21 15:28:08,2161,2,0.004156
3853140,979,2016-04-21 15:28:09,2161,2,0.007849
3853141,979,2016-04-21 15:28:09,8564,76,0.02834


time: 81.9 ms


In [10]:
working_usage.loc[3852720:3853143]

,uid,timestamp,loc,app_id,traffic
3852720,979,2016-04-21 15:03:34,8564,1606,0.0013
3852721,979,2016-04-21 15:03:36,8564,114,0.004419
3852722,979,2016-04-21 15:04:18,8564,114,0.001293
3852723,979,2016-04-21 15:04:21,8564,5,0.001729
3852724,979,2016-04-21 15:04:22,8564,5,0.05744
...,...,...,...,...,...
3853139,979,2016-04-21 15:28:08,8564,2,0.004156
3853140,979,2016-04-21 15:28:09,8564,2,0.007849
3853141,979,2016-04-21 15:28:09,8564,76,0.02834
3853142,979,2016-04-21 15:28:10,8564,2,0.001162


time: 21.2 ms


In [19]:
usage.loc[55664:55690]

,uid,timestamp,loc,app_id,traffic
55664,13,2016-04-25 17:23:17,668,2,0.00188
55665,13,2016-04-25 17:23:18,1709,1,0.000493
55666,13,2016-04-25 17:23:19,668,2,0.000456
55667,13,2016-04-25 17:23:19,668,522,0.001839
55668,13,2016-04-25 17:23:20,668,2,0.001877
55669,13,2016-04-25 17:23:22,1709,1,0.002339
55670,13,2016-04-25 17:23:22,668,522,0.000861
55671,13,2016-04-25 17:23:23,668,522,0.002504
55672,13,2016-04-25 17:23:24,668,1434,0.001634
55673,13,2016-04-25 17:23:24,668,2,0.001374


time: 19.8 ms


In [13]:
working_usage.loc[55664:55686]

,uid,timestamp,loc,app_id,traffic
55664,13,2016-04-25 17:23:17,668,2,0.00188
55665,13,2016-04-25 17:23:18,668,1,0.000493
55666,13,2016-04-25 17:23:19,668,2,0.000456
55667,13,2016-04-25 17:23:19,668,522,0.001839
55668,13,2016-04-25 17:23:20,668,2,0.001877
55669,13,2016-04-25 17:23:22,668,1,0.002339
55670,13,2016-04-25 17:23:22,668,522,0.000861
55671,13,2016-04-25 17:23:23,668,522,0.002504
55672,13,2016-04-25 17:23:24,668,1434,0.001634
55673,13,2016-04-25 17:23:24,668,2,0.001374


time: 29.2 ms


In [16]:
# df_osc_list.to_csv('index_5_jump_5_secs.txt', header=False, index=False, sep=',')

time: 179 ms


In [42]:
user_list = usage['uid'].unique()
base_list = usage['loc'].unique()
all_user = {}
for user in user_list:
    df = usage[usage['uid'] == user]
    loc = df['loc'].unique()
    all_user[user] = len(base_list) - len(loc)
    # if comp.all():
        # all_user.append(user)

time: 4min 51s


In [44]:
dict(sorted(all_user.items(), key=lambda item: item[1]))

{'942': 2015,
 '772': 5497,
 '837': 6261,
 '667': 6270,
 '836': 6371,
 '783': 6377,
 '615': 6401,
 '712': 6440,
 '13': 6457,
 '723': 6474,
 '211': 6532,
 '205': 6546,
 '627': 6551,
 '670': 6557,
 '832': 6557,
 '442': 6565,
 '817': 6586,
 '979': 6590,
 '209': 6593,
 '798': 6593,
 '999': 6598,
 '662': 6602,
 '982': 6602,
 '673': 6606,
 '764': 6606,
 '782': 6606,
 '571': 6622,
 '11': 6623,
 '371': 6623,
 '802': 6623,
 '952': 6624,
 '943': 6628,
 '983': 6628,
 '213': 6634,
 '815': 6634,
 '77': 6636,
 '343': 6638,
 '80': 6639,
 '755': 6641,
 '78': 6641,
 '352': 6642,
 '893': 6642,
 '47': 6643,
 '955': 6643,
 '998': 6643,
 '655': 6644,
 '271': 6645,
 '805': 6645,
 '435': 6646,
 '446': 6647,
 '842': 6647,
 '931': 6648,
 '790': 6649,
 '828': 6649,
 '975': 6650,
 '416': 6652,
 '441': 6652,
 '756': 6652,
 '22': 6654,
 '321': 6654,
 '691': 6654,
 '769': 6655,
 '830': 6655,
 '9': 6655,
 '925': 6655,
 '206': 6656,
 '41': 6656,
 '419': 6656,
 '731': 6656,
 '422': 6658,
 '429': 6658,
 '565': 6658,
 '

time: 51.1 ms


In [48]:
len(base_list)

6739

time: 4.49 ms
